- Export ERP to text (universal)
- Create `mne.EvokedArray()` object 

### Plotting single-subject averaged ERPs

In [ ]:
import os
import mne
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def parse_bdf(BDF_txt):
    f1 = open(BDF_txt, encoding='utf-8')
    f2 = f1.read().split()
    f1.close()
    bin_labels = [f2[2+4*i] for i in range(int((len(f2))/4))]
    
    bins = {}
    for i,x in enumerate(bin_labels):
        bins[x] = i
    return bins

In [ ]:
bins = parse_bdf('S1_bdf_18bins.txt')
bins

In [ ]:
pwd()

In [ ]:
prefix, path = 'S1_18bins_', 'avg_erp_bins'
files_orig = [[f for f in fs if f.startswith(prefix)] for root, dirs, fs in os.walk(path, topdown=True)][0]
files = [None]*len(files_orig)

for f in files_orig:
    try:
        files[bins[f[10:-5]]] = f  # 10 and -5 depend on the file names
    except KeyError:
        files[bins[f[10:-4]]] = f  # 10 and -4 depend on the file names

files

In [ ]:
raw = mne.io.read_raw_eeglab('S1.set')
ch_names = raw.ch_names
ch_names = [i if 'Z' not in i else i[:-1]+'z' for i in ch_names]
ch_names[ch_names.index('FP1')], ch_names[ch_names.index('FP2')] = 'Fp1','Fp2'
to_exclude = ['M1','M2','HEO','VEO']
# M1, M2 are not EOG; just setting them to EOG so that they don't get shown in the topo 
ch_types = ['eeg' if i not in to_exclude else 'eog' for i in ch_names]
info = mne.create_info(ch_names, ch_types=ch_types, sfreq=1000)
# Using biosemi64 so that the electrodes won't exceed the scalp boundary
info.set_montage('biosemi64', on_missing='ignore');

In [ ]:
evokeds, data = [None]*len(files), np.empty((len(files)), dtype=np.ndarray)
tmin = -0.2

os.chdir(path)
for i, f in enumerate(files):
    df = pd.read_csv(f, sep='	').drop(columns=['time','Unnamed: 35'])
    eeglab_data = df.to_numpy().transpose()
    data[i] = eeglab_data
    evokeds[i] = mne.EvokedArray(data=eeglab_data, info=info, tmin=tmin)
os.chdir('..')

evokeds[0]

In [ ]:
display(df)

In [ ]:
ch_dict = {}
for i, x in enumerate(ch_names):
    ch_dict[x] = i
    
ch_to_plot = ['Fz', 'Cz', 'Pz']
bins_to_plot = ['LVF_G_Cor', 'RVF_G_Cor', 'LVF_UG_Cor', 'RVF_UG_Cor']

t = [i for i in range(-200, 1200)]
linestyles = ['solid', 'solid', 'dashed', 'dashed']
colors = ['navy', 'deepskyblue', 'navy', 'deepskyblue']

figure, axes = plt.subplots(3,1, figsize=(12, 15), sharey=True)
for ax, ch in zip(axes.copy().flatten(), ch_to_plot):
    for i, x in enumerate(bins_to_plot):
        ax.plot(t, data[bins[x]][ch_dict[ch]],
                linestyle=linestyles[i], color=colors[i], label=x)
    ax.axvline(x=0, color='black', linewidth=0.5)
    ax.axhline(y=0, color='black', linewidth=0.5)
    ax.set_title(ch)
    ax.set_xlabel('Time (ms)')
    ax.set_xlim(-200, 1200)
    ax.set_xticks(list(range(-200,1201,200)))  # time ticks
    ax.set_ylabel('µV')
    ax.set_ylim(-10, 10)
    ax.set_yticks(list(range(-10,10,5)))  # y ticks
    ax.invert_yaxis()
    ax.yaxis.set_tick_params(labelbottom=True)
    hdl, lbl = ax.get_legend_handles_labels()
ax.legend(hdl, lbl, bbox_to_anchor=[-0.05, 0.00001]) 
figure.tight_layout()
#figure.savefig('S1_plots.png')
plt.show()

In [ ]:
savefig, filepath = False, 'S1.png'

chs_to_plot = evokeds[0].ch_names
pos_dict = {'HEO':1,'Fp1':2,'Fp2':4,'VEO':5,
            'F7':8,'F3':9,'Fz':10,'F4':11,'F8':12,
            'FT7':15,'FC3':16,'FCz':17,'FC4':18,'FT8':19,
            'T7':22,'C3':23,'Cz':24,'C4':25,'T8':26,
            'M1':28,'TP7':29,'CP3':30,'CPz':31,'CP4':32,'TP8':33,'M2':34,
            'P7':36,'P3':37,'Pz':38,'P4':39,'P8':40,
            'O1':44,'Oz':45,'O2':46}
keys, vals = list(pos_dict.keys()), list(pos_dict.values())

nrows, ncols = 7, 7
figsize = (45,35)
figure, axes = plt.subplots(nrows, ncols, figsize=figsize, sharey=True)
for (m,n), ax in np.ndenumerate(axes):
    if m*ncols+n == 45: ax.set_xlabel('Time (ms)', fontsize=30, labelpad=35.0)
    if m*ncols+n == 28: ax.set_ylabel('µV', fontsize=30, labelpad=35.0)
    try:
        if keys[vals.index(m*ncols+n)] not in chs_to_plot: ax.remove()
    except ValueError:
        ax.remove()
            
for ch in chs_to_plot:
    ax = plt.subplot(nrows, ncols, pos_dict[ch]+1)
    for i, x in enumerate(bins_to_plot):
        ax.plot(t, data[bins[x]][ch_dict[ch]], linestyle=linestyles[i], color=colors[i], label=x)
    ax.axvline(x=0, color='black', linewidth=0.5)
    ax.axhline(y=0, color='black', linewidth=0.5)
    ax.set_title(ch, fontsize=40)
    ax.set_xlim(-200,1200)  # time range to plot
    ax.set_xticks([-200,0,300,600,900,1200])  # time ticks
    ax.set_ylim(-20, 20)  # y scale
    ax.set_yticks(list(range(-20,20,10)))  # y ticks
    ax.invert_yaxis()  # negative up
    ax.yaxis.set_tick_params(labelbottom=True, labelsize=20)
    ax.xaxis.set_tick_params(labelbottom=True, labelsize=20)
    hdl, lbl = ax.get_legend_handles_labels()
figure.legend(hdl, lbl, loc='lower left', bbox_to_anchor=[-0.001, -0.001], fontsize=45, borderpad=1)
figure.tight_layout(pad=3)
if savefig == True: plt.savefig(filepath)
plt.show()

### Difference waves: (Ungrammatical Correct) - (Grammatical Correct)

In [ ]:
diff_ug_g = mne.combine_evoked([evokeds[bins['UG_Cor']], evokeds[bins['G_Cor']]], weights=[1,-1])
diff_data = diff_ug_g.get_data()

In [ ]:
ch_to_plot = ['Fz', 'Cz', 'Pz']

figure, axes = plt.subplots(1,3, figsize=(12, 3), sharey=True)
for ax, ch in zip(axes.copy().flatten(), ch_to_plot):
    ax.plot(t, diff_data[ch_dict[ch]],
                linestyle='solid', color='darkgray', label='UG/Cor - G/Cor')
    ax.axvline(x=0, color='black', linewidth=0.5)
    ax.axhline(y=0, color='black', linewidth=0.5)
    ax.set_title(ch)
    ax.set_xlabel('Time (ms)')
    ax.set_xlim(-200, 1200)
    ax.set_ylabel('µV')
    ax.invert_yaxis()
    ax.yaxis.set_tick_params(labelbottom=True)
    hdl, lbl = ax.get_legend_handles_labels()
figure.legend(hdl, lbl, loc='upper left', bbox_to_anchor=[-0.001, 0.001])
figure.tight_layout()
plt.show()

### Difference waves: (Contralateral) - (Ipsilateral)

In [ ]:
to_compare = ['RVF_UG_Cor', 'LVF_UG_Cor']
contra_ipsi_evokeds = [mne.channels.combine_channels(evokeds[bins[to_compare[i]]], 
                                                     groups={'P3-P4': [ch_dict['P3'], ch_dict['P4']],
                                                             'P4-P3': [ch_dict['P4'], ch_dict['P3']]},
                                           method=lambda data: data[0]-data[1]) for i in range(len(to_compare))]
contra_ipsi_evokeds[0].ch_names

In [ ]:
n_times = evokeds[0].get_data().shape[1]
contra_ipsi_data = np.array([contra_ipsi_evokeds[i].get_data() for i in range(len(contra_ipsi_evokeds))],
                            dtype=object).reshape(2, 2, n_times)

ch_to_plot = ['P3-P4', 'P4-P3']
bins_to_plot = ['RVF_UG_Cor', 'LVF_UG_Cor']
colors = ['red', 'black']

figure, axes = plt.subplots(1,2, figsize=(12, 7))
for ax, ch in zip(axes.copy().flatten(), ch_to_plot):
    for i, x in enumerate(bins_to_plot):
        ax.plot(t, contra_ipsi_data[i][ch_to_plot.index(ch)],
                linestyle='solid', color=colors[i], label=x)
    ax.axvline(x=0, color='black', linewidth=0.5)
    ax.axhline(y=0, color='black', linewidth=0.5)
    ax.set_title(ch)
    ax.set_xlabel('Time (ms)')
    ax.set_ylim(-10, 10)
    ax.set_xlim(-200, 1200)
    ax.set_ylabel('µV')
    ax.invert_yaxis() 
    ax.yaxis.set_tick_params(labelbottom=True)
    hdl, lbl = ax.get_legend_handles_labels()
figure.legend(hdl, lbl, loc='upper left', borderpad=2, bbox_to_anchor=[-0.001, 0.001])
figure.tight_layout()
plt.show()

### Topo maps: (Ungrammatical Correct) - (Grammatical Correct)

In [ ]:
diff_data = 1e-06*diff_ug_g.get_data()[:,200:] # Cutting off the 200-ms baseline; scale by 1e-06
evoked_topo = mne.EvokedArray(data=diff_data, info=info)

montage = evoked_topo.get_montage()
montage.plot();

In [ ]:
# N400 (300ms - 500ms)
evoked_topo.plot_topomap(times=np.array([325+i*50 for i in range(0,6)])/1000, average=[0.05]*6,
                         vlim=(-1.8,1.8), time_unit='ms', size=2, res=128, mask=None, contours=5);

In [ ]:
# P600 (700ms - 1000ms)
evoked_topo.plot_topomap(times=np.array([725+i*50 for i in range(0,6)])/1000, average=[0.05]*6,
                          time_unit='ms', vlim=(-1.8, 1.8), size=2, res=128, mask=None, contours=5);

### Exercise: Grand Average? 